In [ ]:
# prompt: pip install pandas scikit-learn tensorflow matplotlib

!pip install pandas scikit-learn tensorflow matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices())

# ==================== ⚙️ Cấu hình GPU ====================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print(f"Đã phát hiện và cấu hình GPU: {gpus[0].name}")
    except RuntimeError as e:
        print("Lỗi khi cấu hình GPU:", e)
else:
    raise RuntimeError("Không phát hiện GPU nào! Vui lòng kiểm tra driver và CUDA.")

In [6]:
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df = df.drop(columns=['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
df = pd.get_dummies(df, drop_first=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                1470 non-null   int64
 1   Attrition                          1470 non-null   int64
 2   DailyRate                          1470 non-null   int64
 3   DistanceFromHome                   1470 non-null   int64
 4   Education                          1470 non-null   int64
 5   EnvironmentSatisfaction            1470 non-null   int64
 6   HourlyRate                         1470 non-null   int64
 7   JobInvolvement                     1470 non-null   int64
 8   JobLevel                           1470 non-null   int64
 9   JobSatisfaction                    1470 non-null   int64
 10  MonthlyIncome                      1470 non-null   int64
 11  MonthlyRate                        1470 non-null   int64
 12  NumCompaniesWorked  

In [ ]:
X = df.drop('Attrition', axis=1)
y = df['Attrition']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:

# 2.2. Xây dựng mô hình
def build_ann_model(input_dim, learning_rate=0.001, activation='relu',
                    hidden_layers=2, units_per_layer=32):
    model = Sequential()
    model.add(Dense(units_per_layer, input_dim=input_dim, activation=activation))

    for _ in range(hidden_layers - 1):
        model.add(Dense(units_per_layer, activation=activation))

    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# 2.3. Đánh giá hiệu suất mô hình cơ bản
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42)

model = build_ann_model(input_dim=X_train.shape[1],
                        learning_rate=0.001,
                        activation='relu',
                        hidden_layers=3,
                        units_per_layer=64)

history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.1, verbose=1)

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
model.save("ann_attrition_model.h5")
print("Mô hình cơ bản đã được lưu vào ann_attrition_model.h5")

In [ ]:
print("\n=== Cross-Validation với 5 Fold ===")
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
for train_index, val_index in kf.split(X_scaled, y):
    X_tr, X_val = X_scaled[train_index], X_scaled[val_index]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    temp_model = build_ann_model(input_dim=X_tr.shape[1],
                                 learning_rate=0.001,
                                 activation='relu',
                                 hidden_layers=2,
                                 units_per_layer=32)

    temp_model.fit(X_tr, y_tr, epochs=30, batch_size=32, verbose=0)
    y_val_pred = (temp_model.predict(X_val) > 0.5).astype("int32")
    acc = accuracy_score(y_val, y_val_pred)
    print(f"Fold {fold} Accuracy: {acc:.4f}")
    fold += 1


In [ ]:
param_grid = {
    'learning_rate': [0.001, 0.005],
    'activation': ['relu', 'tanh'],
    'hidden_layers': [2, 3],
    'units_per_layer': [32, 64]
}

grid = list(ParameterGrid(param_grid))
print(f"\n=== Grid Search: {len(grid)} combinations ===")

best_acc = 0
best_params = None
best_model = None

accuracies = []
param_descriptions = []
precisions = []
recalls = []
f1s = []

for i, params in enumerate(grid):
    print(f"\n[{i+1}/{len(grid)}] Đang thử: {params}")

    model = build_ann_model(
        input_dim=X_train.shape[1],
        learning_rate=params['learning_rate'],
        activation=params['activation'],
        hidden_layers=params['hidden_layers'],
        units_per_layer=params['units_per_layer']
    )

    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, validation_split=0.1)
    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1score = f1_score(y_test, y_pred)

    desc = f"lr={params['learning_rate']}, act={params['activation']}, hl={params['hidden_layers']}, units={params['units_per_layer']}"
    param_descriptions.append(desc)
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1score)

    if acc > best_acc:
        best_acc = acc
        best_params = params
        best_model = model

In [ ]:
best_model.save("best_ann_model.h5")
print("\nMô hình tốt nhất đã được lưu vào: best_ann_model.h5")

# Vẽ biểu đồ so sánh các cấu hình
plt.figure(figsize=(12, 6))
plt.barh(param_descriptions, accuracies, color='skyblue')
plt.xlabel("Accuracy")
plt.title("So sánh các cấu hình Grid Search")
plt.tight_layout()
plt.savefig("gridsearch_comparison.png")
plt.show()

In [ ]:
results_df = pd.DataFrame({
    'Params': param_descriptions,
    'Accuracy': accuracies,
    'Precision': precisions,
    'Recall': recalls,
    'F1-Score': f1s
})

results_df.to_csv("gridsearch_results.csv", index=False)
print("Đã lưu kết quả Grid Search chi tiết vào: gridsearch_results.csv")